# Requirements

In [ ]:

!pip install -q --upgrade accelerate transformers torch bitsandbytes datasets
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "xformers<0.0.26" trl peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are

# Loading model

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch



torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Unused kwargs: ['bnb_8bit_quant_type', 'bnb_8bit_use_double_quant', 'bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Connect to Drive

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
%cd 'drive/MyDrive/Projects/Dastpokht/comment_analyzer/data'

Mounted at /content/drive
/content/drive/MyDrive/Projects/Dastpokht/comment_analyzer/data


# Loading Data

In [ ]:
df = pd.read_csv("./analyzer_result.csv", encoding='utf-8')
print(f"Shape of my Data: {df.shape}\n")
df.head()

Shape of my Data: (5152, 4)



,comment_text,qualities,services,foods
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""سان...","{""{\""service_rate\"": \""5\"", \""part_of_comment\...","{""{\""product_id\"": 18749645, \""product_title\""..."
1,عالي بود,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""است...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748885, \""product_title\""..."
2,خیلی خوب بود ممنون,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""آمر...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748883, \""product_title\""..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,"{""{\""food_rate\"": \""Nan\"", \""food_title\"": \""ن...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18749778, \""product_title\""..."
4,خیلی خوب بود,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""آمر...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748883, \""product_title\""..."


# Preprocessing Data

In [ ]:
import json
import re
import ast
comment_text_array = df['comment_text'].to_numpy()
foods_text_array = df['foods'].to_numpy()
qualities_text_array = df['qualities'].to_numpy()
services_text_array = df['services'].to_numpy()
primary_inputs = []
primary_results = []
for comment, food, qualities, services in zip(comment_text_array, foods_text_array, qualities_text_array, services_text_array):
    json_food = ast.literal_eval(food)
    json_qualities = ast.literal_eval(qualities)
    json_services = ast.literal_eval(services)
    # Print the converted dictionary
    current_title = ""
    current_materials = ""
    foods = list()
    for product , quality in zip(json_food, json_qualities):
        json_product = ast.literal_eval(product)
        json_quality = ast.literal_eval(quality)
        current_materials = current_materials +" , "+ "{"+ json_product['product_title']+ " : " +json_product['product_description']+"}"
        current_title = foods.append(re.sub(r"\u200c", ' ', json_product['product_title']))
    primary_inputs.append({
            "comment": re.sub(r"\u200c", ' ', comment),
            "materials": re.sub(r"\u200c", ' ', current_materials),
            "foods": foods
        })
    primary_results.append({
            "comment_text": comment,
            "order": current_title,
            "response_q": json_qualities,
            "response_s": json_services
        })

In [ ]:
primary_inputs[5147]

{'comment': 'گوشتش خام بود و کامل نپخته بود ، سیب زمینی ها افتضاح بودن',
 'materials': ' , {ماشروم استودیو برگر : یک عدد برگر دست ساز گوشت گوساله 150 گرمی،  پوره قارچ، سس خامه ای قارچ، پنیر چدار، سیب زمینی ترد رشته ای، نان گرد، به همراه سیب زمینی }',
 'foods': ['ماشروم استودیو برگر']}

# Saving Prepared Data

In [ ]:
import csv
def write_data_to_csv(data, filename, fieldnames):
    # Open the file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a writer object specifying the field names
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        # Write the header row
        writer.writeheader()
        # Write data rows
        for row in data:
            writer.writerow(row)
# Call the function with your data and desired file name
write_data_to_csv(primary_inputs, 'primary_inputs.csv', ['comment', 'materials', 'foods'])
write_data_to_csv(primary_results, 'primary_results.csv', ['comment_text', 'order', 'response_q',"response_s"])
a = pd.read_csv("./primary_inputs.csv", encoding='utf-8')
a.head()

,comment,materials,foods
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", {ساندویچ رست بیف چدار لارج : گرم گوشت سردس...",", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ..."
1,عالي بود,", {استیک برگر : برگر دست ساز گوشت استیک راسته...",", [ استیک برگر ] , [ نوشابه قوطی زیرو ]"
2,خیلی خوب بود ممنون,", {آمریکانا برگر : یک عدد برگر دست ساز گوشت ر...",", [ آمریکانا برگر ]"
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", {جوسی برگر : یک عدد برگر دست ساز گوشت ران گ...",", [ جوسی برگر ] , [ نوشابه قوطی کوکا کولا ..."
4,خیلی خوب بود,", {آمریکانا برگر : یک عدد برگر دست ساز گوشت ر...",", [ آمریکانا برگر ] , [ ساندویچ رست چیکن ]..."


In [ ]:
primary_results[5147]

{'comment_text': 'گوشتش خام بود و کامل نپخته بود ، سیب\u200cزمینی\u200cها افتضاح بودن',
 'order': ' ,  [ ماشروم استودیو برگر ] ',
 'response_q': {'{"food_rate": "0", "food_title": "ماشروم استودیو برگر"}'},
 'response_s': {'{"service_rate": "Nan", "part_of_comment": "Nan", "service_classified": "Nan"}'}}

In [ ]:
import json

def convert_ints_to_strings(obj):
    if isinstance(obj, int):
        return str(obj)
    if isinstance(obj, list):
        return [convert_ints_to_strings(item) for item in obj]
    if isinstance(obj, dict):
        return {key: convert_ints_to_strings(value) for key, value in obj.items()}
    return obj

# Your JSON string
json_str = '{"foods overall rating": 1, "qualities": [{"food_title": "\u0645\u0627\u0634\u0631\u0648\u0645 \u0627\u0633\u062a\u0648\u062f\u06cc\u0648 \u0628\u0631\u06af\u0631", "food_rate": 0}, {"food_title": "\u0633\u06cc\u0628\u200c\u0632\u0645\u06cc\u0646\u06cc\u200c\u0647\u0627", "food_rate": 0}], "services": []}'

# Load the JSON data
data = json.loads(json_str)

# Convert integers to strings
data = convert_ints_to_strings(data)

# Dump the data back to a JSON string
json_str = json.dumps(data)

print(json_str)


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
test = """--------------------------------------------------
Analysis:

1. The UserComment is in Persian and expresses dissatisfaction with the food quality.

2. The comment specifically mentions that the meat was raw ("خام بود") and the "سیب‌زمینی‌ها" (potatoes) were awful ("افتضاح بودن").

3. According to the rules, we need to assign a low score for the food items mentioned due to the raw meat and the negative sentiment towards the potatoes.

4. Since the comment does not mention any specific service category directly, we will not assign any service ratings.


JSON Format:

```json

{

    "foods overall rating": 1,

    "qualities": [

        {

            "food_title": "ماشروم استودیو برگر",

            "food_rate": 0

        },

        {

            "food_title": "سیب‌زمینی‌ها",

            "food_rate": 0

        }

    ],

    "services": []

}

```<|end|>"""

In [ ]:

def postprocess(qualities, services, inputs, food_overall):
    food_titels = list()
    for item in qualities:
        food_titels.append(item["food_title"])
    for index,food in enumerate(qualities):
        if food["food_title"] not in inputs["foods"]:
            #remove the food from the qualities
            print(food)
            qualities.pop(index)
    for food in inputs["foods"]:
        if food not in food_titels:
            #add the food to the qualities
            temp_dict = dict()
            temp_dict["food_title"] = food
            temp_dict["food_rate"] = food_overall
            qualities.append(temp_dict)
            food_titels.append(food)

    for index, service in enumerate(services):
        if service["service_category"] not in ["Delivery time", "Quality of packaging", "Cost of packaging"]:
            services.pop(index)

    return qualities, services


def separate_json(text):
    start = text.find('{')
    end = text.rfind('}') + 1
    json_text = text[start:end]
    return json_text

def split_json(json_text):
        data = json.loads(json_text)
        print(data)
        qualities = data.get('qualities', [])
        services = data.get('services', [])
        food_overall = data.get('foods overall rating', None)
        return qualities, services, food_overall

def process(primary_inputs, row):
        json_text = separate_json(primary_inputs)
        qualities, services, food_overall = split_json(json_text)
        qualities, services = postprocess(qualities, services, row, food_overall)
        return qualities, services










In [ ]:
process(test, row=primary_inputs[5147])

{'foods overall rating': 1, 'qualities': [{'food_title': 'ماشروم استودیو برگر', 'food_rate': 0}, {'food_title': 'سیب\u200cزمینی\u200cها', 'food_rate': 0}], 'services': []}
{'food_title': 'سیب\u200cزمینی\u200cها', 'food_rate': 0}


([{'food_title': 'ماشروم استودیو برگر', 'food_rate': 0}], [])

In [ ]:
primary_inputs[5147]["foods"].split(",")

[' ', '  [ ماشروم استودیو برگر ] ']

#Main Prompt

## some of the Changes Made

1. **Adding Warm and Cold in Persian to the Prompt**: The prompt has been updated to include the terms for 'warm' and 'cold' in Persian. This change is intended to make the prompt more understandable for Persian-speaking users.

2. **Fixing the Confusion of NaN Values**: Previously, the presence of NaN values in the data could cause confusion in the analysis. To address this, we have defined overall qualities for food and services. This change helps to provide a clearer picture of the data, even in the presence of NaN values.

3. **Asking the Model to Explain the Analysis for Self Learning**: The model has been updated to provide explanations for its analysis. This feature is intended to promote self-learning by allowing users to understand the reasoning behind the model's output.

original_prompt=
```python
"""Suppose that we are working in the domain of a food delivery app.
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':
UserOrder: {order}

After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):
UserComment: {comment}
As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

If available, here is the list of ingredients for each food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}


Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment, and assign a score per each food and services delivered (an integer between 0 and 5) considering the following rules:

1. Rate Food:
   All the food titles mentioned in the UserOrder, must each receive a score (an integer between 0 and 5), considering the 'Handle Mixed Sentiments' rule (rule number 4). First, decide whether the UserComment is anything related to user's satisfaction about the quality of any food item in UserOrder.
   For this purpose, check if the UserComment directly shares sentiments about specific foods or food ingredients. If it contains sentiments about the food, then rate that food based on the UserComment. If it contains sentiments about the food ingredients,
   then check the food or foods containing those ingredients, utilizing the FoodIngredients list that contains each food with its ingredients and rate that food based on the UserComment.
   if the user has commented the overall food experience and the comment does not specify which food item the rating is for, you can use UserComment to rate all the food items ordered.
   If UserComment does not include any specific review for a specific food item of the UserOrder, set the corresponding food score to 'Nan'.

2. Separate Service Quality Comments:
   The UserComment may contain sentiments about each of the foods or the services, or both. First, decide whether the UserComment is anything related to these predefined service_classified categorie: '[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered]'.
   If the UserComment is related to one or more of these service_classified categorie, then extract and analyze parts of UserComment that pertain specifically to those service_classified categorie (separating them from comments about food quality), and attribute those extracted parts of the comment to part_of_comment.
   After that, assign a score (an integer between 0 and 5) to that service based on part_of_comment.
   If a service-related UserComment doesn’t fit any category or if the sentiment about the service is unclear, set 'part_of_comment', 'service_rate', and 'service_classified' to 'Nan'. Note that there might be more than one service mentioned in the comment.

3. Evaluate Food Temperature:
   If UserComment mentions the temperature of the food, identify which food is being discussed and determine whether it is typically served hot or cold.
   For example, fast foods and main dishes are often served hot, while most drinks and salads are served cold.
   Then, evaluate if the UserComment indicates a satisfactory temperature. For example, the model should asign a low rate for a dish that is mentioned to be 'cold' in the UserComment, but it is often served hot.
   But id the meal is often served 'cold' and it is mentioned to be 'cold' in the UserComment, the meal should gets a high rate.

4. Handle Mixed Sentiments:
   If UserComment contains mixed sentiments (e.g., positive followed by negative), consider the overall tone of the comment at its conclusion. Focus on how the UserComment ends to determine if it’s generally positive or negative. This prevents confusion over mixed messages within the comment.

*ATTENTION1*: As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
*ATTENTION2*: In this step, if UserComment is a general positive or negative UserComment and does not specify any particular food item, apply it to all food items ordered.
For example: Having UserComment='همه چی عالی', all the food items ordered will get a 5 score.


Based on the analysis, generate JSON for foods and services in this format:
 {{
    qualities: [
        {{
            food_title: <food_title>,
            food_rate: <food_rate>
        }}
    ],
    services: [
        {{
            part_of_comment: <part_of_comment>,
            service_rate: <service_rate>,
            service_classified: <service_classified>
        }}
    ]
}}
```
Output only the JSON. Do not output any extra information or explanations."""


### Best prompt

In [ ]:

prompt_enhancements =""""Suppose that we are working in the domain of a food delivery app.

the input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks

UserComment expresses the user's satisfaction with the food quality and services delivered.

Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:

Analyze a user's comment to rate each food item in the UserOrder and the overall food experience.

Task:

1. Determine if the comment expresses satisfaction or dissatisfaction with the quality of any food item in the UserOrder.
2. Check if the comment directly mentions specific foods or food ingredients.
3. If the comment mentions a specific food from the UserOrder, rate that food based on the sentiment expressed.
4. If the comment mentions food ingredients, identify the food(s) containing those ingredients using the FoodIngredients list and rate that food based on the sentiment.
5. If the comment provides an overall assessment of the food experience without specifying a particular food item, only rate the "foods overall rating" score as the general order rating.
6. If the comment does not mention a specific food item, set the score for that food item to 'Nan'.
7. Apply the 'Handle Mixed Sentiments' rule when rating each food item and the overall food experience.

note : "foods overall rating" is not the average value of the food items


Important: if the comment says that a food item was raw (in persian:  خام  ) consider a low score for that food item. also  افتضاح  means awful.


Note: Ensure that each food item in the UserOrder receives a score, and an additional rating is provided for the "foods overall rating".


2. rate Service Quality :

Analyze a user's comment to rate  service categories:


1. Staff Behavior : Look for mentions of driver, delivery person,

2. Delivery Time: Analyze the comment for any mention of time related words or phrases  as "late", "fast", "on time", "delayed", "soon"  If found, assign a score.

3. Food Temperature: Look for words indicating temperature such as "hot", "cold", "warm", Assign a score based on the sentiment expressed about the food temperature

4. Delivered Food not as Ordered : Look for phrases indicating the food was not as ordered such as "wrong order", "not what I ordered", etc. Assign a score based on the sentiment expressed

5. packaging : Identify comments that mention the condition of the packaging when it arrived. Keywords might include “well-packed”, “leaking”, or “damaged”.

6. Cost of Delivery: Look for any mention of the delivery fee or cost. Phrases like “expensive delivery”, “cheap delivery”, or “free delivery” can be relevant.

7. Cost per Value of Foods: This involves comments about whether the user felt they got their money’s worth for the food they ordered. Relevant phrases could be ( "expensive"= " گران " ) “worth the price”, “overpriced”, or “good value”.


Important: food quality is not  part of the service ratings.

Remember, if the comment doesn’t mention services or fit any category, or if the sentiment is unclear, keep the service part empty in the JSON.
do not add any new categories to the service ratings.

3. Evaluate Food Temperature:

Analyze the UserComment to evaluate the temperature of the food.

Task:

1. Check if the UserComment mentions the temperature of a specific food item.
2. Identify the food item being discussed and determine its typical serving temperature (hot or cold).
3. Evaluate the UserComment to determine if the food temperature was satisfactory.
	If the food is typically served hot, but the comment indicates it was ' سرد ' (cold) assign a low temperature score.
	If the food is typically served warm, and the comment indicates it was ' گرم ' (warm), assign a higher temperature score.
	If the food temperature matches its typical serving temperature, assign a higher temperature score.
4. Update the services ratings with the evaluated food temperature score.

Note: Consider the typical serving temperature of different food types, such as hot for fast foods and main dishes, and cold for drinks and salads.


ATTENTION :
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
don't translate food names


Based on the analysis, generate JSON for foods and services in this format:

{{
    "foods overall rating": <foods overall quality rating>,

    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],


    services: [
        {{
            {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_category": <service_category>
        }}
        }}

    ]
}}

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: {order}

If available, here is the  of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {ingredients}

here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {comment}
```

the "food_title" must be in the UserOrder and not in english

Explain the analysis first and then provide the JSON format.  """




system_role = "You are a helpful digital assisstant having brilliant analytical skills."


In [ ]:
ch="""2. Separate Service Quality Comments:

Analyze a user's comment to identify sentiments related to specific service categories. The only categories are:

1. Delivery time
2. Quality of packaging
3. Cost of packaging
4. Cost per value of foods
5. Staff behavior
6. Food temperature
7. Delivered food not as ordered
8. Services overall rating

Important: Do not add any new categories to the service ratings.

Task:

1. Determine if the comment relates to one or more of these service categories.
2. Extract the relevant parts of the comment that pertain to each service category (separating them from food quality comments).
3. Assign a score to each service category based on the extracted comment.
4. If the comment doesn't fit any category or the sentiment is unclear, set the part_of_comment and service_rate to 'Nan'.

Note: A single comment may mention multiple service categories.



"""

In [ ]:
prompt_enhancements

'"Suppose that we are working in the domain of a food delivery app.\n\nthe input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks\n\nUserComment expresses the user\'s satisfaction with the food quality and services delivered.\n\nHaving the context provided,\nyour task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:\n\nHandle Mixed Sentiments:\nIf UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.\nFocus on how the UserComment ends to determine if it’s generally positive or negative.\nThis prevents confusion over mixed messages within the comment.\n\n1. Rate Food quality:\n\nAnalyze a user\'s comment to rate each food item in the UserOrder and the overall food experience.\n\nTask:\n\

In [ ]:
torch.cuda.empty_cache()


In [ ]:

for index, row in enumerate(primary_inputs[5147:5152]):
    #print(row)
    prompt = prompt_enhancements.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=2000)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    inputs.to("cpu")
    outputs.to("cpu")
    torch.cuda.empty_cache()
    #print(messages)
    print("-"*50)
    print(text)
    #result.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)

--------------------------------------------------
Analysis:

1. The UserComment is in Persian and expresses dissatisfaction with the food quality.

2. The comment specifically mentions that the meat was raw ("خام بود") and the "سیب‌زمینی‌ها" (potatoes) were awful ("افتضاح بودن").

3. According to the rules, we need to assign a low score for the food items mentioned due to the raw meat and the negative sentiment towards the potatoes.

4. Since the comment does not mention any specific service category directly, we will not assign any service ratings.


JSON Format:

```json

{

    "foods overall rating": 1,

    qualities: [

        {

            "food_title": "ماشروم استودیو برگر",

            "food_rate": 0

        },

        {

            "food_title": "سیب‌زمینی‌ها",

            "food_rate": 0

        }

    ],

    services: []

}

```<|end|>
0 Done.
**************************************************
--------------------------------------------------
To analyze the given 

In [ ]:
#seperate the json from text
import json
import re

def extract_json(text):
    json_pattern = r"(\{(?:.|\n)*?\})"
    matches = re.findall(json_pattern, text)
    return matches

print(extract_json(text))


['{\n    "foods overall quality rating": "Nan",\n    "qualities": [\n        {\n            "food_title": "برگر کلاسیک",\n            "food_rate": "Nan"\n        }', '{\n            "food_title": "شات پنیر دیپ چدار",\n            "food_rate": "Nan"\n        }', '{\n            "food_title": "سیب زمینی سرخ کرده",\n            "food_rate": "Nan"\n        }', '{\n            "food_title": "ساندویچ زینگر نرمال",\n            "food_rate": "Nan"\n        }']


In [ ]:
a = """"Suppose that we are working in the domain of a food delivery app.

the input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks

UserComment expresses the user's satisfaction with the food quality and services delivered.

Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:

Analyze a user's comment to rate each food item in the UserOrder and the overall food experience.

Task:

1. Determine if the comment expresses satisfaction or dissatisfaction with the quality of any food item in the UserOrder.
2. Check if the comment directly mentions specific foods or food ingredients.
3. If the comment mentions a specific food from the UserOrder, rate that food based on the sentiment expressed.
4. If the comment mentions food ingredients, identify the food(s) containing those ingredients using the FoodIngredients list and rate that food based on the sentiment.
5. If the comment provides an overall assessment of the food experience without specifying a particular food item, only rate the "foods overall rating" score as the general order rating.
6. If the comment does not mention a specific food item, set the score for that food item to 'Nan'.
7. Apply the 'Handle Mixed Sentiments' rule when rating each food item and the overall food experience.

note : "foods overall rating" is not the average value of the food items


Important: if the comment says that a food item was raw (in persian:  خام  ) consider a low score for that food item. also  افتضاح  means awful.


Note: Ensure that each food item in the UserOrder receives a score, and an additional rating is provided for the "foods overall rating".


2. rate Service Quality :

Analyze a user's comment to rate  service categories:


1. Staff Behavior : Look for mentions of driver, delivery person,

2. Delivery Time: Analyze the comment for any mention of time related words or phrases  as "late", "fast", "on time", "delayed", "soon"  If found, assign a score.

3. Food Temperature: Look for words indicating temperature such as "hot", "cold", "warm", Assign a score based on the sentiment expressed about the food temperature

4. Delivered Food not as Ordered : Look for phrases indicating the food was not as ordered such as "wrong order", "not what I ordered", etc. Assign a score based on the sentiment expressed

5. packaging : Identify comments that mention the condition of the packaging when it arrived. Keywords might include “well-packed”, “leaking”, or “damaged”.

6. Cost of Delivery: Look for any mention of the delivery fee or cost. Phrases like “expensive delivery”, “cheap delivery”, or “free delivery” can be relevant.

7. Cost per Value of Foods: This involves comments about whether the user felt they got their money’s worth for the food they ordered. Relevant phrases could be ( "expensive"= " گران " ) “worth the price”, “overpriced”, or “good value”.


Important: food quality is not  part of the service ratings.

Remember, if the comment doesn’t mention services or fit any category, or if the sentiment is unclear, keep the service part empty in the JSON.
do not add any new categories to the service ratings.

3. Evaluate Food Temperature:

Analyze the UserComment to evaluate the temperature of the food.

Task:

1. Check if the UserComment mentions the temperature of a specific food item.
2. Identify the food item being discussed and determine its typical serving temperature (hot or cold).
3. Evaluate the UserComment to determine if the food temperature was satisfactory.
	If the food is typically served hot, but the comment indicates it was ' سرد ' (cold) assign a low temperature score.
	If the food is typically served warm, and the comment indicates it was ' گرم ' (warm), assign a higher temperature score.
	If the food temperature matches its typical serving temperature, assign a higher temperature score.
4. Update the services ratings with the evaluated food temperature score.

Note: Consider the typical serving temperature of different food types, such as hot for fast foods and main dishes, and cold for drinks and salads.


ATTENTION :
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
don't translate food names


Based on the analysis, generate JSON for foods and services in this format:

{{
    "foods overall rating": <foods overall quality rating>,

    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],


    services: [
        {{
            {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_category": <service_category>
        }}
        }}

    ]
}}

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: {order}

If available, here is the  of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {ingredients}

here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {comment}
```

the "food_title" must be in the UserOrder and not in english


Explain the analysis first and then provide the JSON format.  """

'"Suppose that we are working in the domain of a food delivery app.\n\nthe input data ( UserOrder, FoodIngredients, UserComment,) is only within the three backticks\n\nUserComment expresses the user\'s satisfaction with the food quality and services delivered.\n\nHaving the context provided,\nyour task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:\n\nHandle Mixed Sentiments:\nIf UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.\nFocus on how the UserComment ends to determine if it’s generally positive or negative.\nThis prevents confusion over mixed messages within the comment.\n\n1. Rate Food quality:\n\nAnalyze a user\'s comment to rate each food item in the UserOrder and the overall food experience.\n\nTask:\n\

In [ ]:
primary_inputs[5]

{'comment': 'برگرهای اینجا معرکست.',
 'materials': ' , {ماءالشعیر قوطی هی دی لیمو :  میلی لیتر} , {آمریکانا برگر : یک عدد برگر دست ساز گوشت ران گوساله  گرمی، پنیر چدار، سیب زمینی کرانچی، کاهو باتاویا، گوجه فرنگی، پیاز کاراملی، نان برگر مخصوص بیف لاین}',
 'foods': ' ,  [ ماءالشعیر قوطی هی دی لیمو ]  ,  [ آمریکانا برگر ] '}

#prompt2

**this prompt is the enhanced version of the main prompt by gpt_4o**

In [ ]:
gpt_o_prompt="""

Suppose we are working in the domain of a food delivery app.

Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: {order}

If available, here is the list of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}

After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {comment}

As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

Your task is to evaluate the food quality and services delivered based on UserComment and assign a score for each food and service delivered (an integer between 0 (most dissatisfied) and 5 (most satisfied)) considering the following rules:

1. Rate Food Quality
Mixed Sentiments: If UserComment contains mixed sentiments (e.g., positive followed by negative), consider the overall tone of the comment at its conclusion.
Food-Specific Comments: If UserComment directly shares sentiments about specific foods or food ingredients, rate that food based on the comment.
Ingredients-Based Comments: If UserComment mentions food ingredients, check the food or foods containing those ingredients using the FoodIngredients list and rate accordingly.
Overall Food Experience: If the comment is about the overall food experience without specifying any particular food item, rate the 'foods overall quality rating'.
No Specific Review: If UserComment does not include any specific review for a particular food item, set the corresponding food item score to 'Nan'.
General Positive/Negative Comment**: If UserComment is a general positive or negative comment without specifying any particular food item, specify the 'foods overall quality rating' score and set other food items' rating to 'Nan'.
Irrelevant Food Mention: If UserComment mentions a food that is not in the UserOrder, do not consider that food for rating.

2. Separate Service Quality Comments
Service Categories: Identify if UserComment is related to these predefined service categories: '[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Staff behavior, Food temperature, Delivered food not as ordered]'.
Service-Specific Comments: Extract and analyze parts of the UserComment that pertain specifically to those service categories and assign a score based on the extracted parts.
Unclear Service Comments: If a service-related comment doesn’t fit any category or if the sentiment about the service is unclear, set 'part_of_comment', 'service_rate', and 'service_classified' for the categories to 'Nan'.
Overall Service Experience: If the comment is about the overall service experience, rate the 'Services overall rating'.

3. Evaluate Food Temperature
Temperature Comments**: If UserComment mentions the temperature of the food, identify which food is being discussed and determine whether it is typically served hot or cold.
Temperature Satisfaction**: Evaluate if the UserComment indicates a satisfactory temperature. For example, if a dish typically served hot is mentioned as 'cold' (in Persian: 'سرد'), assign a low temperature score in the services ratings.

Attention Points
Persian Language: Be mindful of any different Persian idioms or slang used and their actual meanings.
General Comments: If UserComment is a general positive or negative comment without specifying any particular food item, specify the 'foods overall quality rating' score and set other food items' rating to 'Nan'.
Service vs. Food Quality: Service ratings should not affect food quality ratings. For example, dissatisfaction with food temperature should not affect the food quality rating if the comment does not specify any particular food item.
Sentiment Expressions: Recognize common positive and negative expressions in Persian to help determine the sentiment in UserComment.
Positive Expressions: ["عالی", "خوشمزه", "بهترین", "فوق العاده", "لذت بردم", "دوست داشتم", "معرکه", "بی نظیر", "شگفت انگیز"," خوش طعم"]
Negative Expressions: ["نپسندیدم", "نامطلوب", "مزه نداشت", "بی کیفیت", "ناراضی", "سوخته", "خام", "بدون نمک", "خیلی شور", "بی مزه"]

JSON Output Format
Based on the analysis, generate JSON for foods and services in this format:

```json
{{
    "qualities": [
        {{
            "food_title": "<food_title>",
            "food_rate": "<food_rate>"
        }}
    ],
    "the foods overall quality rating": "<foods overall quality rating>",
    "services": [
        {{
            "part_of_comment": "<part_of_comment>",
            "service_rate": "<service_rate>",
            "service_classified": "<service_classified>"
        }}
    ],
    "the Services overall rating": "<Services overall quality rating>"
}}
```

Explanation:
Explain the analysis and then provide the JSON format.
"""

#prompt3

**same as the main prompt, but without explanation(json_only).**

In [ ]:
primary_inputs[5150]

{'comment': 'واقعا برگرهاتون حرف نداره ،فقط برای این کیفیت نون استفاده شده یکم ضعیفه،اگر نون شیرین و فست فودی نبود به جرات بهترین میشدید',
 'materials': ' , {اولد فشن برگر : یک عدد برگر دست ساز گوشت گوساله  گرمی، سس دست ساز مورگن، خیار ملس ( سوییت پیکل)، کاهو فرانسوی، پیاز، گوجه فرنگی، نان گرد به همراه سیب زمینی}',
 'foods': ' ,  [ اولد فشن برگر ] '}

In [ ]:
prompt_3 = """
Suppose that we are working in the domain of a food delivery app.

the input data is within the three backticks.



Having the context provided,
your task is to evaluate the food quality and services delivered based on UserComment and assign a score per each food and services delivered (an integer between 0 (for most disatisfies) and 5 (for most satisfied))considering the following rules:

Handle Mixed Sentiments:
If UserComment contains mixed sentiments (like positive followed by negative), consider the overall tone of the comment at its conclusion.
Focus on how the UserComment ends to determine if it’s generally positive or negative.
This prevents confusion over mixed messages within the comment.

1. Rate Food quality:
All the food titles mentioned in the UserOrder must each receive a score and an additional rating named 'foods overall rating',
considering the 'Handle Mixed Sentiments' rule.


First, decide whether the UserComment is related to the user's satisfaction about the quality of any food item in UserOrder.
For this purpose, check if the UserComment directly shares sentiments about specific foods or food ingredients.
If it contains sentiments about the food, then rate that food based on the UserComment.
If it contains sentiments about the food ingredients, then check the food or foods containing those ingredients,
utilizing the FoodIngredients list that contains each food with its ingredients and rate that food based on the UserComment.
If the user has commented on the overall food experience and the comment does not specify which food item the rating is for,
you can use UserComment to just rate the 'foods overall quality rating'
If UserComment does not include any specific review for a specific food item of the UserOrder,
set the corresponding food item score to 'Nan'.

2. Separate Service Quality Comments:
The UserComment may contain sentiments about each of the foods or the services, or both.
First, decide whether the UserComment is related to these predefined service_classified categories:
'[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Staff behavior, Food temperature, Delivered food not as ordered]'.
If the UserComment is related to one or more of these service_classified categories,
then extract and analyze parts of the UserComment that pertain specifically to those service_classified categories
(separating them from comments about food quality) and attribute those extracted parts of the comment to part_of_comment. After that,
assign a score to that service based on part_of_comment.
If a service-related UserComment doesn’t fit any category or if the sentiment about the service is unclear, set 'part_of_comment', 'service_rate', and 'service_classified' for the categories to 'Nan'
then you can use UserComment to rate the 'Services overall rating'
Note that there might be more than one service mentioned in the comment.


3. Evaluate Food Temperature:
If the UserComment mentions the temperature of the food,
identify which food is being discussed and determine whether it is typically served hot or cold
For example, fast foods and main dishes are often served hot, while most drinks and salads are served cold.
Then, evaluate if the UserComment indicates a satisfactory temperature. For example,
if dish is often served warm or hot but it is mentioned to be 'cold (in Persian: ' سرد ')' in the UserComment  the model should assign a low teprature score in the services ratings,
But if the meal is often served 'warm' and it is mentioned to be 'warm (in Persian: ' گرم ' in the UserComment,
the order temperature should get a higher score in the services ratings.


ATTENTION1:
As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
but do not translate the UserComment into English.
Sentiment Expressions: Recognize common positive and negative expressions in Persian to help determine the sentiment in UserComment.
Positive Expressions: ["عالی", "خوشمزه", "بهترین", "فوق العاده", "لذت بردم", "دوست داشتم", "معرکه", "بی نظیر", "شگفت انگیز"," خوش طعم"]
Negative Expressions: ["نپسندیدم", "نامطلوب", "مزه نداشت", "بی کیفیت", "ناراضی", "سوخته", "خام", "بدون نمک", "خیلی شور", "بی مزه"]


ATTENTION2:
In this step, if UserComment is a general positive or negative UserComment and does not specify any particular food item,
just specify the 'foods overall quality rating'score, and set other food items' rating to NAN.

ATTENTION3:
the services rating shoud not effect the food quality rating.
for example if the user is disatisfied with the food tempreture and The UserComment does not specify any particular food item but rather gives a general statement about the food temperature,
this statement should not affect the food quality rating.



Based on the analysis, generate JSON for foods and services in this format:


{{
    qualities: [
        {{
            "food_title": <food_title>,
            "food_rate": <food_rate>
        }}
    ],

    the foods overall quality rating: <foods overall quality rating>

    services: [
        {{
            "part_of_comment": <part_of_comment>,
            "service_rate": <service_rate>,
            "service_classified": <service_classified>
        }}
    ]
    the Services overall rating: <Services overall quality rating>
}}

Output only the JSON. Do not output any extra information or explanations

```
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':

UserOrder: {order}

If available, here is the list of ingredients for each or some food included in the UserOrder (called 'FoodIngredients'):
FoodIngredientsFoodIngredients: {ingredients}

After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):

UserComment: {comment}

As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.
```."""


#Finding tricky samples in the dataset

In [ ]:
i = 30
print(result["comment_text"][i])
print(50*"-")
print(result["order"][i])
print(50*"-")
print(result["response_s"][i])
print(50*"-")
print(result["response_q"][i])

NameError: name 'result' is not defined

# Generating Response

#Testing diffrent generate function's parameters

**i tried to find the optimal value for tempreture and top_p**

in some cases the results were beriliant with 0.1 temp and .90 top_p
but cused halucenation for some inputs

In [ ]:
for index, row in enumerate(primary_inputs[5147:5151]):
    print(row)
    prompt = prompt_enhancements.format(order=row['foods'], comment=re.sub(r'\b\w*\d\w*\b', "", row['comment']), ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, max_new_tokens=1000, temperature=0.1, do_sample=True, top_p=0.9)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    inputs.to("cpu")
    outputs.to("cpu")
    print(messages)
    print("-"*50)
    print(text)
    #result.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)

{'comment': 'گوشتش خام بود و کامل نپخته بود ، سیب\u200cزمینی\u200cها افتضاح بودن', 'materials': ' , {ماشروم استودیو برگر : یک عدد برگر دست ساز گوشت گوساله  گرمی،  پوره قارچ، سس خامه ای قارچ، پنیر چدار، سیب زمینی ترد رشته ای، نان گرد، به همراه سیب زمینی }', 'foods': ' ,  [ ماشروم استودیو برگر ] '}
[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': '"Suppose that we are working in the domain of a food delivery app.\n\nHere is the list of foods and beverages ordered by a user, let\'s call it \'UserOrder\':\n\nUserOrder:  ,  [ ماشروم استودیو برگر ] \n\nIf available, here is the list of ingredients for each or some food included in the UserOrder (called \'FoodIngredients\'):\nFoodIngredientsFoodIngredients:  , {ماشروم استودیو برگر : یک عدد برگر دست ساز گوشت گوساله  گرمی،  پوره قارچ، سس خامه ای قارچ، پنیر چدار، سیب زمینی ترد رشته ای، نان گرد، به همراه سیب زمینی }\n\nAfter receiving the \'UserOrder\', here is

KeyboardInterrupt: 

#eval with main prompt

In [ ]:
def evaluate_samples(sample_indices, primary_inputs, prompt_enhancements, system_role, tokenizer, model, temp=False):
    for index in sample_indices:
        prompt = prompt_enhancements.format(order=primary_inputs[index]['foods'],
                                             comment=primary_inputs[index]['comment'],
                                             ingredients=primary_inputs[index]['materials'])
        messages = [{"role": "system", "content": system_role},
                    {"role": "user", "content": prompt}]

        inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
        if not temp:
            outputs = model.generate(inputs, max_new_tokens=1000)
        else:
            outputs = model.generate(inputs, max_new_tokens=1000, temperature=0.1, do_sample=True, top_p=0.9)


        text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
        inputs.to("cpu")
        outputs.to("cpu")

        print(messages)
        print("-"*50)
        print(text)
        print(f"{index} Done.")
        print("*"*50)

# Now you can call the function for each set of samples
easy_samples = [1,2, 4]
regular_samples = [0, 5150, 5140]
tricky_samples = [30, 5147, 3, 35, 5142]

evaluate_samples(easy_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model)
evaluate_samples(regular_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model)
evaluate_samples(tricky_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model)


[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': '"Suppose that we are working in the domain of a food delivery app.\n\nHere is the list of foods and beverages ordered by a user, let\'s call it \'UserOrder\':\n\nUserOrder:  ,  [ استیک برگر ]  ,  [ نوشابه قوطی زیرو ] \n\nIf available, here is the list of ingredients for each or some food included in the UserOrder (called \'FoodIngredients\'):\nFoodIngredientsFoodIngredients:  , {استیک برگر : برگر دست ساز گوشت استیک راسته گوساله  گرمی، پیاز کریسپی، پنیر چدار دوبل، نان برگر مخصوص بیف لاین} , {نوشابه قوطی زیرو :  میلی لیتر}\n\nAfter receiving the \'UserOrder\', here is the user\'s comment (review) about their experience with UserOrder (called \'UserComment\'):\n\nUserComment: عالي بود\n\nAs you see, UserComment expresses the user\'s satisfaction with the food quality and services delivered.\n\n\nHaving the context provided,\nyour task is to evaluate the 

#Eval with gpt_o_prompt

In [ ]:
evaluate_samples(easy_samples, primary_inputs, gpt_o_prompt, system_role, tokenizer, model)
evaluate_samples(regular_samples, primary_inputs, gpt_o_prompt, system_role, tokenizer, model)
evaluate_samples(tricky_samples, primary_inputs, gpt_o_prompt, system_role, tokenizer, model)

[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': '"Suppose that we are working in the domain of a food delivery app.\n\nHere is the list of foods and beverages ordered by a user, let\'s call it \'UserOrder\':\n\nUserOrder:  ,  [ استیک برگر ]  ,  [ نوشابه قوطی زیرو ] \n\nIf available, here is the list of ingredients for each or some food included in the UserOrder (called \'FoodIngredients\'):\nFoodIngredientsFoodIngredients:  , {استیک برگر : برگر دست ساز گوشت استیک راسته گوساله  گرمی، پیاز کریسپی، پنیر چدار دوبل، نان برگر مخصوص بیف لاین} , {نوشابه قوطی زیرو :  میلی لیتر}\n\nAfter receiving the \'UserOrder\', here is the user\'s comment (review) about their experience with UserOrder (called \'UserComment\'):\n\nUserComment: عالي بود\n\nAs you see, UserComment expresses the user\'s satisfaction with the food quality and services delivered.\n\n\nHaving the context provided,\nyour task is to evaluate the 

#Eval with prompt_3(json_only)

In [ ]:
evaluate_samples(easy_samples, primary_inputs, prompt_3, system_role, tokenizer, model)
evaluate_samples(regular_samples, primary_inputs, prompt_3, system_role, tokenizer, model)
evaluate_samples(tricky_samples, primary_inputs, prompt_3, system_role, tokenizer, model)

[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': '\nSuppose that we are working in the domain of a food delivery app.\n\nHere is the list of foods and beverages ordered by a user, let\'s call it \'UserOrder\':\n\nUserOrder:  ,  [ استیک برگر ]  ,  [ نوشابه قوطی زیرو ] \n\nIf available, here is the list of ingredients for each or some food included in the UserOrder (called \'FoodIngredients\'):\nFoodIngredientsFoodIngredients:  , {استیک برگر : برگر دست ساز گوشت استیک راسته گوساله  گرمی، پیاز کریسپی، پنیر چدار دوبل، نان برگر مخصوص بیف لاین} , {نوشابه قوطی زیرو :  میلی لیتر}\n\nAfter receiving the \'UserOrder\', here is the user\'s comment (review) about their experience with UserOrder (called \'UserComment\'):\n\nUserComment: عالي بود\n\nAs you see, UserComment expresses the user\'s satisfaction with the food quality and services delivered.\n\n\nHaving the context provided,\nyour task is to evaluate the

#Eval with temperuture

In [ ]:

evaluate_samples(easy_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model, temp=True)
evaluate_samples(regular_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model, temp=True)
evaluate_samples(tricky_samples, primary_inputs, prompt_enhancements, system_role, tokenizer, model, temp=True)


#phi3 generated prompt

In [ ]:
self_prompt="""
"Given a list of food items, their ingredients, and a comment, generate a JSON output that includes the food titles,
their rates (with "NAN" for unknown), the overall food quality rating based on the comment's sentiment,
and detailed service aspects with their rates and classifications. Ensure the JSON structure is clear,
logical, and efficiently organized.

Food items: {order}
Ingredients: {ingredients}
Comment: {comment}
parse Input Data: Carefully analyze the input data to separate food items, their ingredients, and the comment.

Extract Relevant Information: Identify and extract the food titles, assign default rates, and determine the overall quality rating based on the comment's sentiment.

Analyze Comment for Service Details: Break down the comment into service-related phrases and assign appropriate rates and classifications.

Construct JSON Structure: Organize the extracted information into a well-structured JSON format, ensuring clarity and logical grouping of data.

Optimize for Efficiency: Ensure the prompt guides the LLM to perform these steps in a concise and efficient manner, minimizing unnecessary steps or data processing.

Generate Output: Generate a JSON output that includes the food titles, their rates, the overall quality rating, and detailed service aspects.
json format:

```json
{{
    "qualities": [
        {{
            "food_title": "<food_title>",
            "food_rate": "<food_rate>"
        }}
    ],
    "the foods overall quality rating": "<foods overall quality rating>",
    "services": [
        {{
            "part_of_comment": "<part_of_comment>",
            "service_rate": "<service_rate>",
            "service_classified": "<service_classified>"
        }}
    ],
    "the Services overall rating": "<Services overall quality rating>"
}}
```
"""

In [ ]:
evaluate_samples(easy_samples, primary_inputs, self_prompt, system_role, tokenizer, model)

[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': '\n"Given a list of food items, their ingredients, and a comment, generate a JSON output that includes the food titles, \ntheir rates (with "NAN" for unknown), the overall food quality rating based on the comment\'s sentiment, \nand detailed service aspects with their rates and classifications. Ensure the JSON structure is clear, \nlogical, and efficiently organized.\n\nFood items:  ,  [ استیک برگر ]  ,  [ نوشابه قوطی زیرو ] \nIngredients:  , {استیک برگر : برگر دست ساز گوشت استیک راسته گوساله  گرمی، پیاز کریسپی، پنیر چدار دوبل، نان برگر مخصوص بیف لاین} , {نوشابه قوطی زیرو :  میلی لیتر}\nComment: عالي بود\nparse Input Data: Carefully analyze the input data to separate food items, their ingredients, and the comment.\n\nExtract Relevant Information: Identify and extract the food titles, assign default rates, and determine the overall quality rating based 

In [ ]:
a

,comment,materials,foods
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", {ساندویچ رست بیف چدار لارج : 240 گرم گوشت س...",", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ..."
1,عالي بود,", {نوشابه قوطی زیرو : 330 میلی لیتر} , {استیک...",", [ نوشابه قوطی زیرو ] , [ استیک برگر ]"
2,خیلی خوب بود ممنون,", {آمریکانا برگر : یک عدد برگر دست ساز گوشت ر...",", [ آمریکانا برگر ]"
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", {جوسی برگر : یک عدد برگر دست ساز گوشت ران گ...",", [ جوسی برگر ] , [ نوشابه قوطی کوکا کولا ..."
4,خیلی خوب بود,", {سیمپل فرایز : سیب زمینی سرخ شده} , {آمریکا...",", [ سیمپل فرایز ] , [ آمریکانا برگر ] , ..."
...,...,...,...
5147,گوشتش خام بود و کامل نپخته بود ، سیب‌زمینی‌ها ...,", {ماشروم استودیو برگر : یک عدد برگر دست ساز ...",", [ ماشروم استودیو برگر ]"
5148,بهترین برگر,", {بیکن برگر : یک عدد برگر دست ساز گوشت گوسال...",", [ بیکن برگر ]"
5149,عالی بود. راضی بودم,", {استیک هاوس برگر : یک عدد برگر دست ساز گوشت...",", [ استیک هاوس برگر ] , [ نوشابه قوطی پپسی..."
5150,واقعا برگرهاتون حرف نداره ،فقط برای این کیفیت ...,", {اولد فشن برگر : یک عدد برگر دست ساز گوشت گ...",", [ اولد فشن برگر ]"


# Evaluate Results

In [ ]:
for i in range(30,43):
    print("comment text: ")
    print(result_copy["comment_text"][i])
    print("Order: ")
    print(result_copy["order"][i])
    print("Model response: ")
    print(result_copy["response"][i])
    print("*"*50)

comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ سیب زمینی سرخ کرده ]  ,  [ شات پنیر دیپ چدار ]  ,  [ برگر کلاسیک ]  ,  [ ساندویچ زینگر نرمال ] 
Model response: 
nan
**************************************************
comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
nan
**************************************************
comment text: 
سرد بود
Order: 
 ,  [ پیتزا چیکن آلفردو ]  ,  [ فیله استریپس سوخاری ] 
Model response: 
nan
**************************************************
comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳تکه‌های قبل نبود. اگر قراره تخفیف که می‌دهید، از حجم غذا کم کنید، لطفا این کار را نکنید چون از اعتبار خودتان کم می‌کنید. آخر چه کاری است؟!
Order: 
 ,  [ مرغ سوخاری (3 تکه) ]  ,  [ نوشابه قوطی فانتا ] 
Model response: 
nan
**************************************************
comment text: 
ترشی کلم رو نزنید.
Order: 
 ,  [ سالاد کلم ]  ,  [ ساندویچ چیکن دونر ]  ,  [ شات سس قارچ ] 
Model response: 
nan
**

# Test with original prompts

In [ ]:
original_prompt = ""Suppose that we are working in the domain of a food delivery app. Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder': UserOrder: {order } After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'): UserComment: {comment} As you see, UserComment expresses the user's satisfaction with the food quality and services delivered. If available, here is the list of ingredients for each food included in the UserOrder (called 'FoodIngredients'): FoodIngredients: {ingredients} Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment, and assign a score per each food and services delivered (an integer between 0 and 5) considering the following rules: Handle Mixed Sentiments: If UserComment contains mixed sentiments (e.g., positive followed by negative), consider the overall tone of the comment at its conclusion. Focus on how the UserComment ends to determine if it’s generally positive or negative. This prevents confusion over mixed messages within the comment. 1. Rate Food: All the food titles mentioned in the UserOrder, must each receive a score (an integer between 0(for most disatisfies) and 5(for most satisfied)) and an aditional rating named 'foods overall rating', considering the 'Handle Mixed Sentiments' rule. First, decide whether the UserComment is anything related to user's satisfaction about the quality of any food item in UserOrder. For this purpose, check if the UserComment directly shares sentiments about specific foods or food ingredients. If it contains sentiments about the food, then rate that food based on the UserComment. If it contains sentiments about the food ingredients, then check the food or foods containing those ingredients, utilizing the FoodIngredients list that contains each food with its ingredients and rate that food based on the UserComment. if the user has commented the overall food experience and the comment does not specify which food item the rating is for, you can use UserComment to rate the foods overall rating If UserComment does not include any specific review for a specific food item of the UserOrder, set the corresponding food score to 'Nan'. 2. Separate Service Quality Comments: The UserComment may contain sentiments about each of the foods or the services, or both. First, decide whether the UserComment is anything related to these predefined service_classified categorie: '[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered]'. If the UserComment is related to one or more of these service_classified categorie, then extract and analyze parts of UserComment that pertain specifically to those service_classified categorie (separating them from comments about food quality), and attribute those extracted parts of the comment to part_of_comment. After that, assign a score (an integer between 0 and 5) to that service based on part_of_comment. If a service-related UserComment doesn’t fit anyategory or if the sentiment about the service is unclear, set 'part_of_comment', 'service_rate', and 'service_classified' to 'Nan'. Note that there might be more than one service mentioned in the comment. 3. Evaluate Food Temperature: If UserComment mentions the temperature of the food, identify which food is being discussed and determine whether it is typically served hot or cold. For example, fast foods and main dishes are often served hot, while most drinks and salads are served cold. Then, evaluate if the UserComment indicates a satisfactory temperature. For example, the model should asign a low rate for a dish that is mentioned to be 'cold (in persian:'سرد')' in the UserComment, but it is often served hot. But id the meal is often served 'cold' and it is mentioned to be 'cold' in the UserComment, the meal should gets a high rate. ATTENTION1: As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings. ATTENTION2: In this step, if UserComment is a general positive or negative UserComment and does not specify any particular food item, just specify the 'foods overall rating' and set other food items rating to NAN For example: Having UserComment='همه چی عالی', foods overall rating of the order will get a 5 score. Based on the analysis, generate JSON for foods and services in this format: {{ qualities: [ {{ food_title: <food_title>, food_rate: <food_rate> }} foods_overall_rating: <foods_overall_rating> ], services: [ {{ part_of_comment: <part_of_comment>, service_rate: <service_rate>, service_classified: <service_classified> }} ] }} ``` Output only the JSON. explain your solution step by step with details."""
Output only the JSON. Do not output any extra information or explanations.

"""


In [ ]:
result3= pd.read_csv("./primary_results.csv", encoding='utf-8')
result3.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]","{'{""food_rate"": ""5"", ""food_title"": ""استیک برگر..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""آمریکانا ب..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...","{'{""food_rate"": ""Nan"", ""food_title"": ""نوشابه ق..."
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ آمریکانا برگر ]...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."


In [ ]:
for index, row in enumerate(primary_inputs[30:42]):
    prompt = original_prompt.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=100)
    text = tokenizer.batch_decode(outputs)[0]
    result3.loc[index, "response"] = text
    print(f"{index} Done.")


0 Done.
1 Done.
2 Done.
3 Done.
4 Done.
5 Done.
6 Done.
7 Done.
8 Done.
9 Done.
10 Done.
11 Done.


In [ ]:
for i in range(30,43):
    print("comment text: ")
    print(result3["comment_text"][i])
    print("Order: ")
    print(result3["order"][i])
    print("Model response: ")
    print(result3["response"][i])
    print("*"*50)

comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ ساندویچ زینگر نرمال ]  ,  [ شات پنیر دیپ چدار ]  ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}', '{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}'}
**************************************************
comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}', '{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}'}
**************************************************
comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳ت

In [ ]:
result3.head(200)

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...",<s><|user|> Suppose that we are working in the...
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]",<s><|user|> Suppose that we are working in the...
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]",<s><|user|> Suppose that we are working in the...
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...",<s><|user|> Suppose that we are working in the...
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ آمریکانا برگر ]...",<s><|user|> Suppose that we are working in the...
...,...,...,...
195,بهترینید,", [ همبرگر ]","{'{""food_rate"": ""5"", ""food_title"": ""همبرگر""}'}"
196,از همه لحاظ عالی,", [ کمبو یامی ]","{'{""food_rate"": ""5"", ""food_title"": ""کمبو یامی""}'}"
197,خوب بود,", [ فرش چیز برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""فرش چیز بر..."
198,عالی بود,", [ همبرگر ]","{'{""food_rate"": ""5"", ""food_title"": ""همبرگر""}'}"


Testing with Main prompt

In [ ]:
system_role = "You are a helpful digital assisstant having brilliant analytical skills."

In [ ]:
main_prompt = """Your task is three things that i mentioned below:
    1_ For each food item, you need to:
        - Assign a rating based on quality of food(an integer between 0 and 5)
    2_ For a comment, you need to:
        - Extract the part of the comment that explain about quality of service.(part_of_comment)
        - Classify service based on this qualities: Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered
        - Assign a rating based on quality of service(an integer between 0 and 5)
        - If the comment was not in service classifier fill part_of_comment, service_rate, service_classified with 'Nan'.
        - The comment might have more than one service.
    3_ Your output must be sorted look like below. Don't use extra information and explanation.
                {{
    qualities: [
        {{
            food_title: 'food_title',
            food_rate: 'food_rate'
        }}
    ],
    services: [
        {{
            part_of_comment: 'part_of_comment',
            service_rate: 'service_rate',
            service_classified: 'service_classified'
        }}
    ]
}}
   There is delivery service and customers order one or many different foods and after receiving their food, they can comment on each food item.
    The food items are named in Persian and listed as follows:: ['name of food','name of food', ...].
    The menu of the restaurant and the ingredients of each food item are sorted as follows:
      [{{'foods name': 'materials'}}, {{'foods name': 'materials'}}, ... ]
    pay attention to materials of foods. customer might comment and criticising about materials of food. foods might doesn't have materials.
    analyze the comment and determine which food item it is about.
    if the customer has commented the overall food experience and the comment does not specify which food item the rating is for, you can use comment for all the food items ordered.
    if the customer has commented some foods poorly and has not mentioned other foods or the overall experience, you must fill rate the other foods with 'Nan'.
    if the customer has commented poorly on some food items and has not mentioned other food items or the overall experience, you must not rate on other food items and fill it in with 'Nan'
    If the customer commented badly on some foods and other parts of the comment were the overall experience, use part of the comment with the overall experience on the other foods. The rate for bad foods must be exactly the same part of the sentences criticizing the foods.
    Pay attention to this two comment. they are rated as 5 for quality of foods: 'خدایییش غذاش خوبه', 'حرف نداره'
    The Data is below within the three backticks.
    ```
    comment : {comment}
    materials : {ingredients}

    foods : {order}
    ```

"""

In [ ]:
result4= pd.read_csv("./primary_results.csv", encoding='utf-8')
result4.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ سیمپل فرایز ] , [ ساندویچ رست بیف چدار...","{'{""food_rate"": ""5"", ""food_title"": ""ساندویچ رس..."
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]","{'{""food_rate"": ""5"", ""food_title"": ""استیک برگر..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""آمریکانا ب..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ نوشابه قوطی کوک...","{'{""food_rate"": ""Nan"", ""food_title"": ""ساندویچ ..."
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ سیمپل فرایز ] ...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."


In [ ]:
for index, row in enumerate(primary_inputs[30:42]):
    prompt = main_prompt.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=2000, temperature=0.0)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    result4.loc[index, "response"] = text
    print(f"{index} Done.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


0 Done.


KeyboardInterrupt: 

In [ ]:
for i in range(30,43):
    print("Comment text: ")
    print(result4["comment_text"][i])
    print("Order: ")
    print(result4["order"][i])
    print("Model response: ")
    print(result4["response"][i])
    print("*"*50)

Comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ ساندویچ زینگر نرمال ]  ,  [ شات پنیر دیپ چدار ]  ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}', '{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}'}
**************************************************
Comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
Comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}', '{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}'}
**************************************************
Comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳ت

In [ ]:

df1= pd.read_json("500.json", encoding="utf-8")
df1.head()

,comment,materials,foods
0,سیب زمینی نداشت وکم بود,[{'ساندویچ ران بریان': '300 گرم ران مرغ بریان،...,[ساندویچ ران بریان]
1,نپخته و خام ارسال شد وقتی غذا حاضر نیست چرا ار...,[{'مرغ بریان کامل': 'یک عدد مرغ بریان کامل 1.3...,[مرغ بریان کامل]
2,افتضاح مرغ و سیب زمینی بو میداد نون هم برام نف...,[{'ران مرغ بریان': 'یک عدد ران مرغ بریان شده 4...,"[ران مرغ بریان, نان باگت, سیب زمینی تنوری]"
3,یک چیز دیگه برام ارسال کردن,[{'مرغ بریان کامل': 'یک عدد مرغ بریان کامل 1.3...,"[مرغ بریان کامل, نوشابه خانواده کوکا کولا]"
4,واقعا بد بود,[{'کتف مرغ بریان': '400 گرم کتف مرغ بریان شده،...,"[کتف مرغ بریان, بال مرغ بریان, مرغ بریان کامل]"


In [ ]:
prompt_K = """
    Your task is three things that i mentioned below:
    1_ For each food item, you need to:
        - Assign a rating based on quality of food(an integer between 0 and 5)
    2_ For a comment, you need to:
        - Extract the part of the comment that explain about quality of service.(part_of_comment)
        - Classify service based on this qualities: Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered
        - Assign a rating based on quality of service(an integer between 0 and 5)
        - If the comment was not in service classifier fill part_of_comment, service_rate, service_classified with 'Nan'.
        - The comment might have more than one service.
    3_ Your output must be sorted look like below. Don't use extra information and explanation.
```
        {{
            qualities: [
                {{
                    food_title: <food_title>,
                    food_rate: <food_rate>
                }}
            ],
            services: [
                {{
                    part_of_comment: <part_of_comment>,
                    service_rate: <service_rate>,
                    service_classified: <service_classified>
                }}
            ]
        }}
```
    The Data is below within the three backticks.
    ```
    comment : {comment}
    materials : {ingredients}
    foods : {order}
    ```
    There is delivery service and customers order one or many different foods and after receiving their food, they can comment on each food item.
    The food items are named in Persian and listed as follows:: ['name of food','name of food', ...].
    The menu of the restaurant and the ingredients of each food item are sorted as follows:
      [{{'foods name': 'materials'}}, {{'foods name': 'materials'}}, ... ]
    pay attention to materials of foods. customer might comment and criticising about materials of food. foods might doesn't have materials.
    analyze the comment and determine which food item it is about.
    if the customer has commented the overall food experience and the comment does not specify which food item the rating is for, you can use comment for all the food items ordered.
    if the customer has commented some foods poorly and has not mentioned other foods or the overall experience, you must fill rate the other foods with 'Nan'.
    if the customer has commented poorly on some food items and has not mentioned other food items or the overall experience, you must not rate on other food items and fill it in with 'Nan'
    If the customer commented badly on some foods and other parts of the comment were the overall experience, use part of the comment with the overall experience on the other foods. The rate for bad foods must be exactly the same part of the sentences criticizing the foods.
    Pay attention to this two comment. they are rated as 5 for quality of foods: 'خدایییش غذاش خوبه', 'حرف نداره'
    """

In [ ]:
result5= pd.read_csv("./primary_results.csv", encoding='utf-8')
result5.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...","{'{""food_rate"": ""5"", ""food_title"": ""ساندویچ رس..."
1,عالي بود,", [ نوشابه قوطی زیرو ] , [ استیک برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""استیک برگر..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""آمریکانا ب..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ نوشابه قوطی کوکا کولا ] , [ جوسی برگر ...","{'{""food_rate"": ""Nan"", ""food_title"": ""ساندویچ ..."
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ سیمپل فرایز ] ...","{'{""food_rate"": ""5"", ""food_title"": ""ساندویچ رس..."


In [ ]:
for index, row in enumerate(primary_inputs[30:40]):
    prompt = prompt_K.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=2000, temperature=0.0)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    print(messages)
    print("-"*50)
    print(text)
    result5.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)

[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': "\n    Your task is three things that i mentioned below:\n    1_ For each food item, you need to:\n        - Assign a rating based on quality of food(an integer between 0 and 5)\n    2_ For a comment, you need to:\n        - Extract the part of the comment that explain about quality of service.(part_of_comment)\n        - Classify service based on this qualities: Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered\n        - Assign a rating based on quality of service(an integer between 0 and 5)\n        - If the comment was not in service classifier fill part_of_comment, service_rate, service_classified with 'Nan'.\n        - The comment might have more than one service.\n    3_ Your output must be sorted look like below. Don't use extra infor

In [ ]:
for i in range(30,40):
    print("Comment text: ")
    print(result5["comment_text"][i])
    print("Order: ")
    print(result5["order"][i])
    print("Model response: ")
    print(result5["response"][i])
    print("*"*50)

Comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ سیب زمینی سرخ کرده ]  ,  [ شات پنیر دیپ چدار ]  ,  [ ساندویچ زینگر نرمال ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}', '{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}'}
**************************************************
Comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
Comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}', '{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}'}
**************************************************
Comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳ت